In [1]:
#*******************************************************************************************
 #
 #  File Name:  PyAviationAccidentsWebScrapeExtract.ipynb
 #
 #  File Description:
 #      This interactive Python notebook, PyAviationAccidentsWebScrapeExtract.ipynb, uses 
 #      the Python modules, Splinter and Beautiful Soup to web scrape aviation accident 
 #      information from the Aviation Safety Network website.
 #
 #
 #  Date            Description                             Programmer
 #  ----------      ------------------------------------    ------------------
 #  10/05/2023      Initial Development                     N. James George
 #
 #******************************************************************************************/

import PyFunctions as function
import PyLogFunctions as log_function
import PyLogSubRoutines as log_subroutine
import PySubRoutines as subroutine

import PyAviationAccidentsConstants as local_constant
import PyAviationAccidentsFunctions as local_function

import pandas as pd

import re

from bs4 import BeautifulSoup
from splinter import Browser

from datetime import datetime as dt
from pymongo import MongoClient

In [2]:
CONSTANT_LOCAL_FILE_NAME \
    = 'PyAviationAccidentsWebScrapeExtract.ipynb'

In [3]:
log_subroutine \
    .SetLogMode \
        (False)

log_subroutine \
    .SetDebugMode \
        (False)

log_subroutine \
    .SetImageMode \
        (False)


log_subroutine \
    .BeginProgramExecution \
        ('PyAviationAccidentsWebScrapeExtract')

## **1.1: Visit and Scrape the [Aviation Safety Network](https://aviation-safety.net)**

In [4]:
nowDatetimeObject = dt.now()

log_subroutine \
    .PrintAndLogWriteText \
        ('\033[1m' \
         + f'WEBPAGE SCRAPING PROCESS BEGINS... ' \
         + f"{nowDatetimeObject.strftime('%H:%M:%S')}" \
         + '\n\n'
         + '\033[0m')

aviationAccidentDictionary \
    = {'accident_id': None,
       'status': None,
       'acc_date': None,
       'acc_time': None,
       'acc_datetime': None,
       'type': None,
       'operator': None,
       'registration': None,
       'msn': None,
       'first_flight': None,
       'crew_fatalities': None,
       'crew_occupants': None,
       'passenger_fatalities': None,
       'passenger_occupants': None,
       'total_fatalities': None,
       'total_occupants': None,
       'aircraft_damage': None,
       'phase': None,
       'nature': None,
       'depart_airport': None,
       'destination_airport': None,
       'flight_number': None,
       'location': None,
       'classifications': None}

aviationAccidentDocumentDictionary \
    = {'accident_id': None, 
       'narrative': None}

aviationAccidentDictionaryList \
    = []

aviationAccidentDocumentDictionaryList \
    = []

# This is the Splinter web driver for the Google Chrome browser.
chromeBrowserSplinterWebDriver \
    = Browser \
        ('chrome')

log_function \
    .DebugReturnObjectWriteObject \
        (chromeBrowserSplinterWebDriver)

accidentIDInteger \
    = 100

# This repetition loop runs from the first year to the last year.
for yearIndexInteger in range (local_constant.BEGIN_YEAR, 
                               local_constant.END_YEAR + 1):
    
    pageIndexInteger = 1

    # This repetition loop runs through all the pages of entries 
    # for each year and has an upper limit of 100 pages.
    while (pageIndexInteger <= 100):
        
        # This String is the URL for the primary webpage of accident 
        # entries.
        primaryDatabaseURLString \
            = local_constant.AVIATION_SAFETY_NET_MAIN_URL \
              + local_constant.AVIATION_SAFETY_NET_DATABASE_PRIMARY_URL \
              + str(yearIndexInteger) \
              + '/' \
              + str(pageIndexInteger)
        
        log_subroutine \
            .PrintAndLogWriteText \
                ('\033[1m' \
                 + f'{primaryDatabaseURLString}\n' \
                 + '\033[0m')
        
        
        # This line of code uses Splinter to visit the current primary 
        # webpage.
        chromeBrowserSplinterWebDriver \
            .visit \
                (primaryDatabaseURLString)
        
        
        # This line of code extracts the HTML from the current primary 
        # webpage and stores it in a String.
        primaryWebPageHTMLStringVariable \
            = chromeBrowserSplinterWebDriver \
                .html

        log_function \
            .DebugReturnObjectWriteObject \
                (primaryWebPageHTMLStringVariable)
        
        
        # This line of code converts the HTML string to a Beautiful Soup 
        # Object for parsing.
        primaryAviationSafetyBeautifulSoupObject \
            = BeautifulSoup \
                (primaryWebPageHTMLStringVariable, 
                 'html.parser')

        
        # This line of code looks for the one table with class, hp.
        primaryAviationAccidentsTableBSElementObject \
            = primaryAviationSafetyBeautifulSoupObject \
                .find \
                    ('table', 
                     class_ = 'hp')

        
        # If the script cannot find a table, then it exits the loop.
        if primaryAviationAccidentsTableBSElementObject == None:
            
            break
        
        
        # This line of code finds all the entries for aviation accidents 
        # in the table.
        tableDataBSResultSetObject \
            = primaryAviationAccidentsTableBSElementObject \
                .find_all \
                    ('tr', 
                     class_ = 'list')
        
        
        # This repetition loop moves through all the aviation accident 
        # entries in the table.
        for rowBSElement in tableDataBSResultSetObject:
           
            # This line of code extracts all the elements in a table row.
            rowBSResultSetObject \
                = rowBSElement \
                    .find_all \
                        ('td')
        
            # These lines of code extract the date of an accident and 
            # converts them to date objects before adding them to a 
            # List.
            accidentDateObject \
                = dt.strptime \
                        (rowBSResultSetObject[0].text, '%d %b %Y') \
                    .date()
            
            
            # This line of code ensures that the entry occurred in the 
            # specified date range.
            if int(accidentDateObject.strftime('%Y')) \
                    < int(local_constant.BEGIN_YEAR) \
                or int(accidentDateObject.strftime('%Y')) \
                        > int(local_constant.END_YEAR):
                
                continue
                
            
            # These lines of code extract the link suffix to the details 
            # webpage for the table entry and creates the URL.
            tempBeautifulSoupObject \
                = BeautifulSoup \
                    (str(rowBSElement))
            
            for element in tempBeautifulSoupObject.find_all ('a', href = True):
                        
                secondaryDatabaseURLSuffixString \
                    = element['href']
            
            secondaryDatabaseURLString \
                = local_constant.AVIATION_SAFETY_NET_MAIN_URL \
                  + secondaryDatabaseURLSuffixString
        
        
            # This line of code uses Splinter to visit the secondary webpage.
            chromeBrowserSplinterWebDriver \
                .visit \
                    (secondaryDatabaseURLString)
            
            
            # This line of code extracts the HTML from the secondary webpage 
            # and stores it in a String variable.
            secondaryWebPageHTMLStringVariable \
                = chromeBrowserSplinterWebDriver \
                    .html
            
            
            # This line of code converts the HTML string to a Beautiful Soup 
            # Object for parsing.
            secondaryAviationSafetyBeautifulSoupObject \
                = BeautifulSoup \
                    (secondaryWebPageHTMLStringVariable, 
                     'html.parser')
            
            # This line of code looks for the first table with class, hp.
            secondaryAviationAccidentsTableBSElementObject \
                = secondaryAviationSafetyBeautifulSoupObject \
                    .find \
                        ('tbody')
            
            
            textElementsStringList \
                = []
            
            # This repetition loop creates a List of all the HTML elements 
            # in a table row.
            for element in secondaryAviationAccidentsTableBSElementObject:
                
                if element.text != '\n':
                    
                    textElementsStringList \
                        .append \
                            (element.text)

            
            # This line of code assigns the unique accident ID to the
            # dataFrame row Dictionary.
            aviationAccidentDictionary \
                ['accident_id'] \
                    = 'ACC' + str(accidentIDInteger)
            
            # This function call returns the status from the webpage 
            # and assigns the value to a Dictionary element.            
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Status:',
                             True))
            
            aviationAccidentDictionary \
                ['status'] \
                    = tempString.strip()
            
            # This line of code assigns the Date object
            # to a Dictionary element.
            aviationAccidentDictionary \
                ['acc_date'] \
                    = accidentDateObject
            
            # This function call returns the date from the webpage and 
            # assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Time:',
                             False))
                
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString:
            
                aviationAccidentDictionary \
                    ['acc_time'] \
                        = tempString.strip()
            else:
                
                aviationAccidentDictionary \
                    ['acc_time'] \
                        = '00:00'
                
            # This function call creates a datetime object from the 
            # scraped information and assigns the value to a Dictionary 
            # element.
            aviationAccidentDictionary \
                ['acc_datetime'] \
                    = local_function \
                        .ReturnDateTimeFromString \
                            (accidentDateObject,
                             aviationAccidentDictionary['acc_time'])

            # This function call returns the aircraft type from the 
            # webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Type:',
                             True))
            
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString:
                
                aviationAccidentDictionary \
                    ['type'] \
                        = tempString.strip()
                
            else:
                
                continue
            
            # This function call returns the aircraft operator
            # from the webpage and assigns the value to a 
            # Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Operator:',
                             True))
                
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString:
                
                aviationAccidentDictionary \
                    ['operator'] \
                        = tempString.strip()
                
            else:
                
                continue
            
            # This function call returns the registration from the 
            # webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Registration:'))
            
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString:
                
                aviationAccidentDictionary \
                    ['registration'] \
                        = tempString.strip()
                
            else:
                
                continue
            
            # This function call returns the manufacturer's serial 
            # number from the webpage and assigns the value to a 
            # Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'MSN:'))

            aviationAccidentDictionary \
                ['msn'] \
                    = tempString.strip()
            
            # This function call returns the aircraft's first flight 
            # year or date from the webpage and assigns the value to 
            # a Dictionary element.
            tempString \
                   = str \
                        (local_function \
                            .ReturnHtmlElementTextFromList \
                                (textElementsStringList,
                                 'First flight:'))
            
            tempString \
                = tempString.strip()
            
            if len(tempString) > 4:
                
                aviationAccidentDictionary \
                    ['first_flight'] \
                        = tempString[0:4] 
            
            else:
            
                aviationAccidentDictionary \
                    ['first_flight'] \
                        = tempString
                         
            # These lines find the crew occupant and fatality numbers 
            # from the webpage and assigns the values to Dictionary 
            # elements.           
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Crew:'))

            resultsIntegerList \
                = local_function \
                    .ReturnFatalitiesAndOccupantsFromString \
                        (tempString)

            aviationAccidentDictionary \
                ['crew_fatalities'] \
                    = resultsIntegerList[0]
                
            aviationAccidentDictionary \
                ['crew_occupants'] \
                    = resultsIntegerList[1]

            # These lines find the passenger occupant and fatality 
            # numbers from the webpage and assigns the values to 
            # Dictionary elements.           
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Passengers:'))
          
            resultsIntegerList \
                = local_function \
                    .ReturnFatalitiesAndOccupantsFromString \
                        (tempString)
            
            aviationAccidentDictionary \
                ['passenger_fatalities'] \
                    = resultsIntegerList[0]
                
            aviationAccidentDictionary \
                ['passenger_occupants'] \
                    = resultsIntegerList[1]
            
            # These lines find the total occupant and fatality 
            # numbers from the webpage and assigns the values to 
            # Dictionary elements.           
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Total:'))

            resultsIntegerList \
                = local_function \
                    .ReturnFatalitiesAndOccupantsFromString \
                        (tempString)
            
            aviationAccidentDictionary \
                ['total_fatalities'] \
                    = resultsIntegerList[0]
                
            aviationAccidentDictionary \
                ['total_occupants'] \
                    = resultsIntegerList[1]
                         
            if aviationAccidentDictionary['crew_occupants'] == 0 \
                and aviationAccidentDictionary['passenger_occupants'] == 0 \
                and aviationAccidentDictionary['total_occupants'] == 0:
                         
                continue

            # This function call returns the aircraft damage from 
            # the webpage and assigns the value to a Dictionary 
            # element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Aircraft damage:'))
                         
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString:
                        
                aviationAccidentDictionary \
                    ['aircraft_damage'] \
                        = tempString.strip()
                         
            else:
                         
                continue
                         
            # This function call returns the flight phase from the 
            # webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Phase:'))
                         
            aviationAccidentDictionary \
                ['phase'] \
                    = tempString.strip()
                         
            # This function call returns the nature of the flight from 
            # the webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Nature:'))
            
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString \
                and '-' not in tempString \
                and '?' not in tempString:
                
                aviationAccidentDictionary \
                    ['nature'] \
                        = tempString.strip()
            
            else:
                         
                continue
                
            # This function call returns the departure airport from 
            # the webpage and assigns the value to a Dictionary 
            # element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Departure airport:'))
                
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString \
                and '-' not in tempString \
                and '?' not in tempString:
            
                aviationAccidentDictionary \
                    ['depart_airport'] \
                         = tempString.strip()
                         
            else:
            
                continue
                
            # This function call returns the destination airport from
            # the webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Destination airport:'))
                
            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString \
                and '-' not in tempString \
                and '?' not in tempString:
                         
                aviationAccidentDictionary \
                    ['destination_airport'] \
                        = tempString.strip()
                         
            else:
                
                continue
            
            # This function call returns the flight number from the 
            # webpage and assigns the value to a Dictionary element.
            tempString \
                = str \
                    (local_function \
                     .ReturnHtmlElementTextFromList \
                        (textElementsStringList,
                         'Flightnumber:'))
                        
            aviationAccidentDictionary \
                ['flight_number'] \
                    = tempString.strip()
                         
            # These lines of code return the accident location from
            # the webpage and assign the value to a Dictionary element.
            tempString \
                = local_function \
                        .ReturnHtmlElementTextFromList \
                            (textElementsStringList,
                             'Location:')
            
            tempString \
                = str \
                    (local_function \
                        .ReturnFormattedAccidentLocationString \
                            (tempString))
                         

            if 'unknown' not in tempString \
                and 'Unknown' not in tempString \
                and 'none' not in tempString \
                and 'None' not in tempString:
                         
                aviationAccidentDictionary \
                    ['location'] \
                        = tempString.strip()
                         
            else:
                
                continue
            
            
            # These lines of code return the accident classifications
            # from the webpage and assign the values to a Dictionary 
            # element.
            classificationsHtmlList \
                = secondaryAviationSafetyBeautifulSoupObject \
                    .findAll \
                        ('a', href = re.compile('/database/event/'))

            aviationAccidentDictionary \
                ['classifications'] \
                    = local_function \
                        .ReturnClassificationsListFromHTML \
                            (classificationsHtmlList)
            
            if len(aviationAccidentDictionary['classifications']) == 0:
                
                aviationAccidentDictionary \
                    ['classifications'] \
                        = 'None'
                
            elif len(aviationAccidentDictionary['classifications']) == 1:
            
                tempString \
                    = aviationAccidentDictionary \
                        ['classifications'][0]
                         
                         
                aviationAccidentDictionary \
                    ['classifications'] \
                        = tempString.strip()

            # These lines of code append the completed accident Dictionary 
            # to a List.
            tempDictionary \
                = aviationAccidentDictionary.copy()
            
            aviationAccidentDictionaryList \
                .append \
                    (tempDictionary)
            
            
            # This line of code assigns the unique accident ID to the
            # document Dictionary.
            aviationAccidentDocumentDictionary \
                ['accident_id'] \
                    = 'ACC' + str(accidentIDInteger)
            
            # These lines of code return the accident narrative from
            # the webpage and assign the value to a document Dictionary 
            # element.
            aviationAccidentDocumentDictionary \
                ['narrative'] \
                    = str \
                        (secondaryAviationSafetyBeautifulSoupObject \
                            .find \
                                ('span', 
                                 lang = 'en-US') \
                            .text)
            
            # These lines of code append the completed document Dictionary 
            # to a List.
            tempDictionary \
                = aviationAccidentDocumentDictionary.copy()
            
            aviationAccidentDocumentDictionaryList \
                .append \
                    (tempDictionary)          
            
            # This line of code increments the unique accident identifier for
            # the next entry
            accidentIDInteger += 1
            
        # This line of code increments the webpage number for a single year.
        pageIndexInteger += 1
        
nowDatetimeObject = dt.now()

log_subroutine \
    .PrintAndLogWriteText \
        ('\033[1m' \
         + f'WEBPAGE SCRAPING PROCESS COMPLETE... ' \
         + f"{nowDatetimeObject.strftime('%H:%M:%S')}" \
         + '\n\n'
         + '\033[0m')

WEBPAGE SCRAPING PROCESS BEGINS... 15:37:08


https://aviation-safety.net/database/year/1972/1

https://aviation-safety.net/database/year/1972/2

https://aviation-safety.net/database/year/1972/3

https://aviation-safety.net/database/year/1972/4

https://aviation-safety.net/database/year/1972/5

https://aviation-safety.net/database/year/1973/1

https://aviation-safety.net/database/year/1973/2

https://aviation-safety.net/database/year/1973/3

https://aviation-safety.net/database/year/1973/4

https://aviation-safety.net/database/year/1974/1

https://aviation-safety.net/database/year/1974/2

https://aviation-safety.net/database/year/1974/3

https://aviation-safety.net/database/year/1974/4

https://aviation-safety.net/database/year/1975/1

https://aviation-safety.net/database/year/1975/2

https://aviation-safety.net/database/year/1975/3

https://aviation-safety.net/database/year/1975/4

https://aviation-safety.net/database/year/1976/1

https://aviation-safety.net/database/year/1976/2

http

https://aviation-safety.net/database/year/2007/1

https://aviation-safety.net/database/year/2007/2

https://aviation-safety.net/database/year/2007/3

https://aviation-safety.net/database/year/2007/4

https://aviation-safety.net/database/year/2008/1

https://aviation-safety.net/database/year/2008/2

https://aviation-safety.net/database/year/2008/3

https://aviation-safety.net/database/year/2008/4

https://aviation-safety.net/database/year/2009/1

https://aviation-safety.net/database/year/2009/2

https://aviation-safety.net/database/year/2009/3

https://aviation-safety.net/database/year/2009/4

https://aviation-safety.net/database/year/2010/1

https://aviation-safety.net/database/year/2010/2

https://aviation-safety.net/database/year/2010/3

https://aviation-safety.net/database/year/2011/1

https://aviation-safety.net/database/year/2011/2

https://aviation-safety.net/database/year/2011/3

https://aviation-safety.net/database/year/2012/1

https://aviation-safety.net/database/year/2012/2



## **1.2: Save to DataFrames**

In [5]:
aviationAccidentsDataFrame \
    = pd.DataFrame \
        .from_dict \
            (aviationAccidentDictionaryList)
    
    
log_function \
    .DebugReturnObjectWriteObject \
        (aviationAccidentsDataFrame)

In [6]:
aviationAccidentsDocumentDataFrame \
    = pd.DataFrame \
        .from_dict \
            (aviationAccidentDocumentDictionaryList)
    
    
log_function \
    .DebugReturnObjectWriteObject \
        (aviationAccidentsDocumentDataFrame)

## **1.3: Display DataFrames**

In [7]:
log_subroutine \
    .PrintAndLogWriteText \
        ('\033[1m' \
         + 'There are {:,} complete aviation accident records from 1972 to 2022.' \
             .format(len(aviationAccidentsDataFrame))
         + '\n\n'
         + '\033[0m')

There are 2,051 complete aviation accident records from 1972 to 2022.




In [8]:
captionString \
    = 'Table 1.3.1: Aviation Accident Scraped Data (1972-2022) - First'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (aviationAccidentsDataFrame.head(5),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

accident_id,status,acc_date,acc_time,acc_datetime,type,operator,registration,msn,first_flight,crew_fatalities,crew_occupants,passenger_fatalities,passenger_occupants,total_fatalities,total_occupants,aircraft_damage,phase,nature,depart_airport,destination_airport,flight_number,location,classifications
ACC100,None,1972-01-07,12:10,1972-01-07 12:10:00,Sud Aviation SE-210 Caravelle VI-R,Iberia,EC-ATV,163,1963,6,6,98,98,104,104,Destroyed,Approach (APR),Domestic Scheduled Passenger,"Valencia Airport (VLC/LEVC), Spain","Ibiza Airport (IBZ/LEIB), Spain",IB602,ca 15 km W of Ibiza Airport (IBZ) (Spain),Controlled Flight Into Terrain (CFIT) - Mountain
ACC101,None,1972-01-09,00:00,1972-01-09 00:00:00,Lockheed L-188A Electra,Air Manila International,PI-C1060,1021,1959,0,4,0,0,0,4,Damaged beyond repair,Unknown (UNK),Training,"Manila International Airport (MNL/RPLL), Philippines","Manila International Airport (MNL/RPLL), Philippines",None,Manila International Airport (MNL) (Philippines),None
ACC102,None,1972-02-01,14:58,1972-02-01 14:58:00,VFW/Fokker VFW.614,VFW-Fokker,D-BABA,G001,1971,1,3,0,0,1,3,Damaged beyond repair,Maneuvering (MNV),Test,"Bremen Airport (BRE/EDDW), Germany","Bremen Airport (BRE/EDDW), Germany",None,near Bremen Airport (BRE) (Germany),None
ACC103,None,1972-02-04,00:00,1972-02-04 00:00:00,de Havilland Canada C-7A Caribou (DHC-4),Air America,393,51,Unkn,1,4,1,34,2,38,Damaged beyond repair,Landing (LDG),Military,"Ban Xon Airstrip, Laos","Pha Khao Airstrip, Laos",None,Pha Khao (Laos),None
ACC104,None,1972-02-27,00:00,1972-02-27 00:00:00,Antonov An-24B,"Aeroflot, North Kavkaz Civil Aviation Directorate",CCCP-46418,87304103,1968,0,4,0,43,0,47,Damaged beyond repair,Approach (APR),Domestic Scheduled Passenger,"Krasnodar Airport (KRR/URKK), Russia","Mineralnye Vody Airport (MRV/URMM), Russia",None,2 km (1.3 mls) NW of Mineralnye Vody Airport (MRV) (Russia),None


In [9]:
captionString \
    = 'Table 1.3.2: Aviation Accident Scraped Data (1972-2022) - Last'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (aviationAccidentsDataFrame.tail(5),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

accident_id,status,acc_date,acc_time,acc_datetime,type,operator,registration,msn,first_flight,crew_fatalities,crew_occupants,passenger_fatalities,passenger_occupants,total_fatalities,total_occupants,aircraft_damage,phase,nature,depart_airport,destination_airport,flight_number,location,classifications
ACC2146,Information verified through authorities or other official sources.,2022-11-18,10:19,2022-11-18 10:19:00,Cessna 208B Grand Caravan EX,Copper Mountain Aviation LLC,N2069B,208B5657,2021,4,4,0,0,4,4,Destroyed,Maneuvering (MNV),Test,"Renton Airport, WA (RNT/KRNT), United States of America","Renton Airport, WA (RNT/KRNT), United States of America",None,"near Snohomish, WA (United States of America)",None
ACC2147,Information verified through authorities or other official sources.,2022-11-29,19:10,2022-11-29 19:10:00,Learjet 45,Jett Aircraft,N988MC,45-352,2007,0,2,0,6,0,8,Substantial,Landing (LDG),Executive,"Waterloo Airport, IA (ALO/KALO), United States of America","Batesville Regional Airport, AR (BVX/KBVX), United States of America",3,"Batesville Regional Airport, AR (BVX) (United States of America)",Runway excursion (overrun)
ACC2148,Information verified through authorities or other official sources.,2022-11-30,08:05,2022-11-30 08:05:00,Learjet 36,Aery Aviation,N12FN,36-016,1975,0,3,0,0,0,3,Substantial,Takeoff (TOF),Training,"Newport News/Williamsburg International Airport, VA (PHF/KPHF), United States of America","Newport News/Williamsburg International Airport, VA (PHF/KPHF), United States of America",None,"Newport News/Williamsburg International Airport, VA (PHF) (United States of America)","['Rejected takeoff', 'Tire failure', 'Runway excursion (overrun)']"
ACC2149,Information verified through authorities or other official sources.,2022-12-15,20:25,2022-12-15 20:25:00,Swearingen SA226-TC Metro II,Key Lime Air,N398KL,TC-398,1981,0,1,0,0,0,1,Substantial,Landing (LDG),Cargo,"Dodge City Municipal Airport, KS (DDC/KDDC), United States of America","Wichita Dwight D. Eisenhower National Airport, KS (ICT/KICT), United States of America",KG147,"Wichita Dwight D. Eisenhower National Airport, KS (ICT) (United States of America)","['Gear-up landing', 'Runway mishap']"
ACC2150,Accident investigation report completed and information captured,2022-12-27,18:29,2022-12-27 18:29:00,Embraer EMB-505 Phenom 300,Skystallion LLC,N16DF,50500167,2013,0,1,0,5,0,6,Substantial,Landing (LDG),Domestic Non Scheduled Passenger,"Jackson Hole Airport, WY (JAC/KJAC), United States of America","Hawthorne Airport, CA (HHR/KHHR), United States of America",None,"Hawthorne Airport, CA (HHR) (United States of America)",Runway excursion (overrun)


In [10]:
captionString \
    = 'Table 1.3.3: Aviation Accident Narratives (1972-2022) - First'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (aviationAccidentsDocumentDataFrame.head(5),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

accident_id,narrative
ACC100,"Iberia flight 602, a Caravelle VI-R passenger jet, was destroyed when it struck a mountainside of the Sierra de Atalayasa, near Ibiza Airport (IBZ), Spain. All 104 on board were killed.The aircraft had departed Madrid (MAD) on a domestic service to Valencia (VLC) and Ibiza Airport (IBZ). While flying an approach to runway 07 at Ibiza, the airplane struck a 415 m high mountain, 30 m below the summit.It was determined that the pilot had failed to maintain the minimum altitude for a visual approach to runway 07"
ACC101,Ran off the end of the runway during a training flight.
ACC102,"Since its first flight on July 14, 1971, the VFW/Fokker VFW.614 prototype was engaged in several test flights. Changes were made following elevator flutter problems during test flights. For the flight on February 1, 1972 the airplane had been fitted with flutter dampers.The flight test called for an asymmetric flutter at 220 kt and 10,000 feet. This time the flutter could not be eliminated by slowing down and the crew abandoned the aircraft. Co-pilot Hans Bardill was killed when his parachute failed to open. The airplane entered a vertical dive and crashed."
ACC103,"The Caribou transport aircraft, carrying Lao military forces to a forward position, was making a normal landing approach when an obstruction was sighted on the runway. The pilot initiated a go-around but the left engine failed to respond and the aircraft's left wing contacted the ground causing the aircraft to pancake into the ground. One Lao Air Freight Specialist and one Lao military passenger were killed.The Caribou, which had been leased from the US Air Force (serial 61-2393), sustained major damage and, after the recovery of several components, was abandoned at the crash site."
ACC104,"The Antonov lost control and crashed on approach to Mineralnye Vody's runway 12 when the pilot unintentionally applied reverse thrust at an altitude of 200 m, about 2000 m short of the runway."


In [11]:
captionString \
    = 'Table 1.3.4: Aviation Accident Narratives (1972-2022) - Last'

currentStylerObject \
    = function \
        .ReturnStylerObjectStandardFormat \
            (aviationAccidentsDocumentDataFrame.tail(5),
             captionString)

log_function \
    .ReturnStylerObjectSavePNGImage \
        (currentStylerObject,
         captionString)

accident_id,narrative
ACC2146,"The Cessna 208B Grand Caravan EX, N2069B, was destroyed when it impacted terrain in Snohomish, Washington. All four onboard were killed.The aircraft was operated by Raisbeck Engineering in a flight testing program to obtain a Supplemental Type Certificate (STC) for an aerodynamic drag reduction system (DRS) on the Cessna 208B EX. The company already held such an STC for the Cessna 208B model. The airplane began flights to support the flight-test three days before the accident. The program started out with flight-test data-collection flights to establish baseline data.The purpose of the accident flight was to complete a test that was cut short the day prior, which was baseline testing of the airplane's aft CG stall characteristics. Preliminary radar track data indicated that after departing Renton, Washington around 09:25, the airplane continued to the north in a gradual climb to about 9,500 ft mean sea level (msl) and began a series of turns/maneuvers. The airplane proceeded for about 45 minutes varying in altitude between about 6,500 ft to 10,275 ft msl. At 10:17 the track data indicated the airplane was climbing to an altitude of 9,700 ft msl and turning to the left, making a near 360° turn and then at 10:19:06, there was a sharp 180° left-turn. The track continued west until the last recorded hit at 10:19:18. During the last 12 seconds the track indicated that the airplanes descent rate exceeded 14,000 ft-per-minute (fpm) and gradually lessened to 8,700 fpm. At the time the flight crew were likely at the second to last maneuver on the test card which specified: 96 kts indicated airspeed; flaps in landing configuration; 930 ft-lbs of torque; propeller rpm fully forward; and accelerated 30° bank to the left.Witnesses reported that they observed the airplane break-up inflight and watched pieces floating down. The airplane then descended in a nose-low near-vertical corkscrew maneuver toward the ground."
ACC2147,"The Learjet 45 operating as Jett Aircraft flight 3 sustained substantial damage subsequent to impact with airport fencing following the runway overrun at Batesville Regional Airport (BVX/KBVX), Arkansas.The two pilots sustained minor injuries and the six passengers were not injured. During the arrival to BVX, a non-towered airport, the air traffic controller cleared the airplane for the RNAV (GPS) Rwy 8 approach and the crew cancelled IFR after visually acquiring the runway environment. The airplane crossed the final approach fix at 265 knots ADS-B groundspeed and the Runway 8 threshold at 190 knots ADS-B groundspeed. The airplane landed about 2,000 ft past the threshold of the 6,022 ft runway, which was wet due to earlier precipitation. Tire marks consistent with intermittent braking application started about 3,021 ft from the end of the runway. Tire marks consistent with continuous anti-skid braking application began about 2,069 ft from the end of the runway and continued until the end of the runway.The airplane exited the runway at an airspeed of about 100 knots ADS-B groundspeed, then continued forward and struck a ditch and the airport perimeter fence 370 m past the runway end besides U.S. Route 167.. During the collision with the fence the forward fuselage sustained substantial damage."
ACC2148,"The Learjet 36 blew a tire on take off from Newport News/Williamsburg International Airport (PHF/LPHF), Virginia.The three occupants were not injured.According to the pilot-in-command (PIC), the preflight inspection, engine start-up, and taxi were normal. During the takeoff roll, he heard ""one boom"" and one second later called V1, which is the maximum speed at which a rejected takeoff can be initiated in the event of an emergency. At the same time, he called ""V1"" the PIC noted a second ""boom"" and the second-in-command, who was the pilot flying, called to abort the takeoff. The PIC reported to air traffic control that they were aborting the takeoff with a suspected blown tire. They atte

## **1.4: Export DataFrames to CSV Files**

In [12]:
aviationAccidentsDataFrame \
    .to_csv \
        (local_constant.AVIATION_ACCIDENTS_DATA_CSV_FILE)

In [13]:
aviationAccidentsDocumentDataFrame \
    .to_csv \
        (local_constant.AVIATION_ACCIDENTS_DOCUMENTS_CSV_FILE)

## **1.5: Close Browser**

In [14]:
chromeBrowserSplinterWebDriver \
    .quit()

In [15]:
#log_subroutine \
#    .EndProgramExecution()